In [1]:
import sys
import tempfile

import h5py
import numpy as np

from versioned_hdf5 import VersionedHDF5File

shape = (17, 17)
chunks = (10, 10)

tmpdir = tempfile.TemporaryDirectory()
path = f"{tmpdir.name}/data.h5"

with h5py.File(path, "w") as f:
    vf = VersionedHDF5File(f)
    with vf.stage_version("r0") as sv:
        sv.create_dataset(
            "value",
            data=np.arange(1, np.prod(shape) + 1).reshape(shape),
            chunks=(chunks),
            maxshape=(None, None),
            fillvalue=1337,
        )

In [2]:
f = h5py.File(path, "r+")
vf = VersionedHDF5File(f)
ctx = vf.stage_version("r1")
sv = ctx.__enter__()
dset = sv["value"]

In [3]:
dset.dataset.staged_changes

StagedChangesArray<shape=(17, 17), chunk_size=(10, 10), dtype=int64, fill_value=1337, 1 base slabs, 0 staged slabs>
slab_indices:
[[1 1]
 [1 1]]
slab_offsets:
[[ 0 10]
 [20 30]]

In [4]:
dset.dataset.staged_changes.load()
dset.dataset.staged_changes

StagedChangesArray<shape=(17, 17), chunk_size=(10, 10), dtype=int64, fill_value=1337, 1 base slabs, 1 staged slabs>
slab_indices:
[[2 2]
 [2 2]]
slab_offsets:
[[ 0 10]
 [20 30]]

In [5]:
for i in range(4):
    print(f['_version_data/value/hash_table'][i]["hash"].tobytes().hex(), end="")
    print("", f['_version_data/value/hash_table'][i]["shape"])

5005acb81cb04e3068acc1bc172cbb8347bd0009d2e0b380b579d0ee976c87d6 [ 0 10]
62529ef56b786159d34d41233205d97293ca624ab89abba777d2757280ebec33 [10 20]
3d3abca5a66f1597b9264e9a1d3f29c1bb40e2aec866e4de372185ed03219c22 [20 27]
8aa5369b8aa4d13e81d975a8dba3ba274346d1ccebfda9c32ac1528df1eb9cad [30 37]


In [6]:
import hashlib
for _, _, chunk in dset.dataset.staged_changes.changes():
    h = hashlib.sha256()
    h.update(np.ascontiguousarray(chunk))
    h.update(bytes(str(chunk.shape), "ascii"))
    print(h.hexdigest(), chunk.shape)
    

5005acb81cb04e3068acc1bc172cbb8347bd0009d2e0b380b579d0ee976c87d6 (10, 10)
62529ef56b786159d34d41233205d97293ca624ab89abba777d2757280ebec33 (10, 7)
3d3abca5a66f1597b9264e9a1d3f29c1bb40e2aec866e4de372185ed03219c22 (7, 10)
8aa5369b8aa4d13e81d975a8dba3ba274346d1ccebfda9c32ac1528df1eb9cad (7, 7)
